<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/simple_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading libraries

In [1]:
import pandas as pd
from tensorflow.keras.layers import *
import tensorflow as tf
import random, os, sys
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
import tensorflow as tf
from google.colab import drive
import os
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from keras.layers import *
from keras.optimizers import *
from keras.models import *
from keras import backend as K
from tensorflow.keras.models import *
import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
import gc

loading drive

In [2]:

drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


Loading data from kaggle

In [3]:

os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

test_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


Loading dataframes

In [4]:

trn=pd.read_csv('/content/gdrive/My Drive/fraud/train.csv')
tst=pd.read_csv('/content/gdrive/My Drive/fraud/test.csv')
ls=list(trn.filter(regex='V'))
trn=trn.drop(ls,1)
tst=tst.drop(ls,1)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (210,222) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Label encoding data

In [5]:



class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)
cols=list(trn.select_dtypes(include=object))
cols.remove('id')
for col in cols:
  le=LabelEncoderExt()
  le.fit(trn[col].astype(str))
  trn[col]=le.transform(trn[col].astype(str))
  tst[col] = tst[col].map(lambda s: '<unknown>' if s not in le.classes_ else s)
  tst[col]=le.transform(tst[col].astype(str))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Getting id features

In [6]:

dum=list(trn.filter(regex='dum'))
cat=cols+['id']
isna=list(trn.filter(regex='isna'))
cat.remove('id')
categorical = ["ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", "addr1", "addr2",
               "P_emaildomain",'P_emaildomain_first','P_emaildomain_second', "R_emaildomain",
              "DeviceInfo", "DeviceType"] + ["id_0" + str(i) for i in range(1, 10)] +\
                ["id_" + str(i) for i in range(10, 39)] + \
                 ["M" + str(i) for i in range(1, 10)]



tot=list(set(dum+cat+isna+['day','month','week']+categorical))
num=[ i for i in list(trn) if i not in tot]

num.remove('id')
num.remove('isFraud')
for col in tqdm(num):
  cols=[col+'_mean',col+'_std']
  trn[col+'_mean']=trn.groupby(['id'])[col].transform('mean')
  trn[col+'_std']=trn.groupby(['id'])[col].transform('std')
  
  tst[col+'_mean']=tst.groupby(['id'])[col].transform('mean')
  tst[col+'_std']=tst.groupby(['id'])[col].transform('std')
  for c in cols:
    mn=trn[c].mean()
    std=trn[c].std()
    trn[c]=(trn[c]-mn)/std
    tst[c]=(tst[c]-mn)/std 
trn=trn.drop(['id'],1)
tst=tst.drop(['id'],1)

num=[ i for i in list(trn) if i not in tot]

100%|██████████| 35/35 [02:12<00:00,  3.79s/it]


Loading autoenc features

In [7]:
autoenc=np.load('/content/gdrive/My Drive/fraud/with_id_withpoutV.npy')
autoenc.shape

(1097231, 256)

Merging data

In [8]:
autoenc=pd.DataFrame(autoenc)
for col in list(autoenc):
  mn=autoenc[col].mean()
  std=autoenc[col].std()
  autoenc[col]=(autoenc[col]-mn)/std  
trn_s=trn.shape[0]
df=pd.concat([trn,tst],0).reset_index(drop=True)
del([trn,tst])
gc.collect()
df=pd.concat([df,autoenc],1)
trn=df.loc[:trn_s-1]
tst=df.loc[trn_s:].reset_index(drop=True)
del([df])
gc.collect()
trn=trn.drop(['Unnamed: 0'],1)
tst=tst.drop(['Unnamed: 0','isFraud'],1)
gc.collect()
trn.head()

,card2_isna,card3_isna,card4_isna,card5_isna,card6_isna,addr1_isna,addr2_isna,dist1_isna,P_emaildomain_isna,D1_isna,D2_isna,D3_isna,D4_isna,D5_isna,D10_isna,D11_isna,D15_isna,M1_isna,M2_isna,M3_isna,M4_isna,M5_isna,M6_isna,M7_isna,M8_isna,M9_isna,M4,P_emaildomain,M9,card3,M5,M2,M6,card6,M7,ProductCD,card4,M3,card5,card2,...,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2,8,3,7,0,1,1,2,3,5,2,1,11,11,...,1.242764,-0.515518,-1.070670,0.283700,-0.665361,-0.852507,-1.188304,0.235176,-0.808980,0.537635,2.922530,-0.106844,-0.310748,2.003823,1.206149,0.013289,-0.451956,-1.558216,-0.052969,0.549370,-0.155466,-0.329755,4.310970,-0.076769,-1.207729,-0.193912,1.231712,0.668962,1.233703,0.913392,0.353394,-0.393244,0.981463,-0.800058,0.091785,0.120459,-0.529338,-1.485394,-0.249264,-0.743785
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0,5,3,7,1,3,1,2,3,5,3,3,0,11,...,-0.478335,0.278179,-0.868818,-1.043651,0.801288,0.078223,-0.640398,-0.337298,0.001820,-0.845505,-0.359985,-0.408767,-0.196191,-0.397315,-0.708109,-0.724816,-0.552898,-0.734606,-0.310484,0.382740,-0.549191,-0.849574,0.198847,-0.259470,0.256196,0.807907,0.377824,0.699971,1.761081,0.309027,-0.165418,-0.400574,0.586140,0.229128,-0.713779,0.102720,-0.190755,0.367235,-0.751962,-0.614538
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,10,0,7,0,1,0,3,0,5,5,1,5,5,...,-0.279917,-0.574741,1.029608,0.457375,-0.584431,-0.589340,2.700197,-0.130066,-0.420789,0.219158,-1.303340,0.558095,-0.327077,1.490529,0.436610,-0.415684,-0.427045,0.136411,-1.589380,-0.439486,1.084522,1.017003,-0.326726,0.418317,0.057420,-0.592301,0.012839,0.497301,-0.945999,0.155201,0.438123,-0.353266,0.837998,0.539504,-0.980364,-0.953120,-0.191095,2.448992,-0.122963,0.135550
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0,11,3,7,1,3,0,3,3,5,3,3,1,11,...,0.790510,-0.270352,-0.006041,-0.073428,-1.076433,0.006397,-0.278142,0.044966,-0.439494,-0.794641,-1.078903,-0.158959,-0.303655,-0.836505,-0.306516,-1.054407,-0.214335,0.060106,0.213959,-0.050495,-0.671942,0.124549,-0.507537,-0.649432,-0.507792,-0.606063,0.177337,0.123741,2.553579,-1.065499,0.097979,-0.297628,-0.708184,0.510937,-0.204277,-0.424759,-0.350952,0.162537,-0.261409,0.150569
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,5,3,7,3,3,3,2,3,1,3,3,0,6,...,-1.040117,0.125807,0.640090,-1.335913,1.071193,-0.229325,-0.147024,-0.218421,0.180540,-0.468195,0.867738,-0.689950,-0.434339,-1.096154,-1.231862,-1.096684,1.020662,-0.510981,-0.023783,-0.609389,0.325300,-0.701391,-0.154755,-0.618930,-0.372212,1.245477,-0.275173,0.924416,-0.024507,-1.042178,2.667389,3.365522,0.725972,-0.521776,-1.169338,0.780491,-0.140265,1.355072,0.048784,-0.236553


Reduce memory useage

In [9]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
trn=reduce_mem_usage(trn)
tst=reduce_mem_usage(tst)

Memory usage of dataframe is 2392.40 MB
Memory usage after optimization is: 604.86 MB
Decreased by 74.7%
Memory usage of dataframe is 2048.85 MB
Memory usage after optimization is: 517.53 MB
Decreased by 74.7%


In [10]:
a=trn.isna().sum()
ls=a[a>0].index
for col in ls:
  trn[col]=trn[col].fillna(trn[col].mean())
  tst[col]=tst[col].fillna(tst[col].mean())
a=trn.isna().sum()
ls=a[a>0].index
for col in ls:
  trn=trn.drop([col],1)
  tst=tst.drop([col],1)

Making the callbacks and loading model

In [ ]:
dk={}
class RocCallback(Callback):
    def __init__(self,validation_data):
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
        self.ep=0

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        self.ep+=1
        if self.ep%10==0:
          y_pred_val = self.model.predict(self.x_val)
          roc_val = roc_auc_score(self.y_val, y_pred_val)
          print('roc-auc_val: %s' % str(round(roc_val,4)))

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return
def load_model():
  K.clear_session()



  inp=Input((523,))
  x=Dense(256,activation=custom_gelu)(inp)
  x=Dropout(0.3)(x)
  x=BatchNormalization()(x)
  x=Dense(256,activation=custom_gelu)(x)
  x=Dropout(0.3)(x)
  x=BatchNormalization()(x)
  x=Dense(256,activation=custom_gelu)(x)
  x=Dropout(0.3)(x)
  x=BatchNormalization()(x)
  x=Dense(1,activation='sigmoid')(x)
  mod=Model(inputs=[inp],outputs=x)
  return mod

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
ls=list(trn['month'].unique())
for en,month in enumerate([(ls[4],ls[5]),(ls[3],ls[4]),(ls[3],ls[5])]):
  train=trn.loc[trn['month']<=month[0]]
  test=trn.loc[(trn['month']>=month[1])&(trn['month']<6)]
  train=train.drop(['month'],1)
  test=test.drop(['month'],1)
  mod=load_model()
  roc = RocCallback(
                  validation_data=(test.drop(['isFraud'],1), test['isFraud']))
  mod.compile(optimizer=Nadam(0.0001),loss='binary_crossentropy')
  es=EarlyStopping(monitor='val_loss',min_delta=0.0001,mode='min',restore_best_weights=True,patience=50)
  mod.fit(train.drop(['isFraud'],1),train['isFraud'],validation_data=(test.drop(['isFraud'],1),test['isFraud']),batch_size=2048,epochs=1000,callbacks=[es,roc])
  del([train,test])
  gc.collect()
  df=trn.loc[trn['month']==ls[6]].reset_index(drop=True).drop(['month'],1)
  pre=mod.predict(df.drop(['isFraud'],1))
  scr=roc_auc_score(df['isFraud'],pre)
  dk[str(scr)]=mod.predict(tst.drop(['month'],1))

Epoch 1/1000
243/243 [==============================] - 2s 9ms/step - loss: 0.6787 - val_loss: 0.4578
Epoch 2/1000
243/243 [==============================] - 2s 7ms/step - loss: 0.4638 - val_loss: 0.3154
Epoch 3/1000
243/243 [==============================] - 2s 7ms/step - loss: 0.3036 - val_loss: 0.2089
Epoch 4/1000
243/243 [==============================] - 2s 7ms/step - loss: 0.2097 - val_loss: 0.1589
Epoch 5/1000
243/243 [==============================] - 2s 7ms/step - loss: 0.1692 - val_loss: 0.1409
Epoch 6/1000
243/243 [==============================] - 2s 7ms/step - loss: 0.1511 - val_loss: 0.1302
Epoch 7/1000
243/243 [==============================] - 2s 7ms/step - loss: 0.1408 - val_loss: 0.1272
Epoch 8/1000
243/243 [==============================] - 2s 7ms/step - loss: 0.1338 - val_loss: 0.1213
Epoch 9/1000
243/243 [==============================] - 2s 7ms/step - loss: 0.1299 - val_loss: 0.1198
Epoch 10/1000
243/243 [==============================] - 5s 19ms/step - loss: 0.12